In [1]:
# Issue one.
# The plane and the implicit widget are not working well together.

In [1]:
# Manually make the widget.
import numpy as np, vtk
import SimPEG as simpeg
import simpegViz
from glob import glob

In [2]:
# Load data - UBC MT models
workDir = '/home/gudni/Dropbox/Work/ISOR/Hengill/MT3D/New3Dinv/TrueNorthSSc/RefineMesh'
M = simpeg.Utils.meshutils.readUBCTensorMesh(workDir + '/reMesh1.txt')

modDict = {}
for mod in glob(workDir+'/run6/inv_*'):
    key = mod.split('/')[-1].replace('.con','')
    modDict[key] = simpeg.Utils.meshutils.readUBCTensorModel(mod,M)

In [3]:
# Setup the viewer
vtkViewer = simpegViz.vtkView(M,{'C':modDict})

In [4]:
# vtkViewer.Show()
from tvtk.api import tvtk
tvtkrect = tvtk.to_tvtk(vtkViewer._cells)

In [7]:
def view3Dvolume(tvtkObj): 
    # Make a widget
    global plane, selectActor
    plane = tvtk.Plane()
    clipper = tvtk.ClipDataSet()
    clipper.input_connection = tvtkObj.producer_port
    clipper.clip_function = plane
    clipper.inside_out
    # Make mapper and actor
    selectMapper = tvtk.DataSetMapper(input_connection = clipper.output_port)
    # The actoer
    selectActor = tvtk.LODActor(mapper = selectMapper)
    selectProp = selectActor.property
    selectProp.representation = 's'
    # Create the RenderWindow, Renderer, RenderWindowInteractor
    ren = tvtk.Renderer()
    ren.add_actor(selectActor)
    ren.background = (1, 1, 1)
    renWin = tvtk.RenderWindow(size=(400,400))
    # For some reason this doesn't work, likely a bug
    #renWin.add_renderer(ren) 
    # This is a work around... Add the renderer in the vtk
    tvtk.to_vtk(renWin).AddRenderer(tvtk.to_vtk(ren))
    iren = tvtk.RenderWindowInteractor(render_window=renWin)
    tvtk.to_vtk(iren).GetInteractorStyle().SetCurrentStyleToTrackballCamera()
    # The callback function
    def movePlane(obj, event):
        global plane, selectActor
        # Don't know why, but obj is a VTK to a TVTK object
        # Use vtk method and convert plane to VTK object
        obj.GetPlane(tvtk.to_vtk(plane))
        selectActor.visibility = 1

    # Associate the line widget with the interactor
    planeWidget = tvtk.ImplicitPlaneWidget()
    planeWidget.interactor = iren
    planeWidget.place_factor = 1.05
    planeWidget.input = clipper.output
    planeWidget.place_widget(tvtkObj.bounds)
    planeWidget.add_observer('InteractionEvent', movePlane)
    planeWidget.enabled = 0 
    planeWidget.outside_bounds = 0
    planeWidget.scale_enabled = 0
    planeWidget.outline_translation = 0
    planeWidget.origin = np.array(tvtkObj.bounds[0::2])
    planeWidget.plane_property.opacity = 0.1
    
    # Direction widget
    nArr = tvtk.ConeSource(direction=(.0,1.0,.0),height=50,radius=10,resolution=100)
    nArrMap = tvtk.PolyDataMapper(input_connection=nArr.output_port)
    nArrAct = tvtk.Actor(mapper=nArrMap)

    oriWid = tvtk.OrientationMarkerWidget(key_press_activation=False)
#     oriWid.viewport = (0.01,0.01,.11,.11)
    oriWid.orientation_marker = nArrAct
    oriWid.interactor = iren
    oriWid.enabled = 1
    oriWid.on()
    # Start interaction.
    
    renWin.render()

    #planeWidget.on()
    iren.initialize()
    iren.start()

    # When done
    iren.terminate_app()
    renWin.finalize()
    del renWin, iren

In [8]:
view3Dvolume(tvtkrect)

In [18]:
propVTK.SetR

NameError: name 'vtkrw' is not defined

In [ ]:
iren.interactor_style.set_current_style_to_trackball_actor